In [ ]:
from pgml import Database
import os
import json

In [ ]:
local_pgml = "postgres://postgres@127.0.0.1:5433/pgml_development"

conninfo = os.environ.get("PGML_CONNECTION",local_pgml)
db = Database(conninfo,min_connections=4)

In [ ]:
collection_name = "test_pgml_sdk_1"
collection = db.create_collection(collection_name)

In [ ]:
from datasets import load_dataset

data = load_dataset("squad", split="train")
data = data.to_pandas()
data.head()

data = data.drop_duplicates(subset=["context"])
print(len(data))
data.head()

documents = [
    {
        'text': r['context'],
        'metadata': {
            'title': r['title']
        }
    } for r in data.to_dict(orient='records')
]
documents[:3]

In [ ]:
collection.upsert_documents(documents[0:200])
collection.generate_chunks()
collection.generate_embeddings()

In [ ]:
results = collection.vector_search("Who won 20 Grammy awards?", top_k=2)
print(json.dumps(results,indent=2))

In [ ]:
# db.delete_collection(collection_name)